# KKBOX Churn Prediction - Data Understanding

**Identifying which files are needed, what the relevant variables mean and creating one clean file to proceed**

*Arabella Specker, Farouq El-Abbass, Fynn Oldenburg, Kevin Nguyen & Vaclav Pasek (Machine Learning 2487-2122_S2)*

-------------------------------------------------------------------------------------------------------------------------------

The following files containing the following variables are available on https://www.kaggle.com/qmdo97/kkboxdataset

![alt text](media/kkbox_data_understanding.png)

All files together amount to over 4GB of storage space. It is therefore required to carefully select the files and variables which are processed. Some of the files include similar or the same attributes e.g. transactions.csv and transactions_v2.csv.

The goal of this phase is to create one clean file which will be used from the Data Preprocessing phase onwords. The variables we will use for this are highlighted in the tables above. We want to conduct a binary classification on the target variable "is_churn" located in the file train_v2.csv which includes data about user chain behaviour in march. All attributes we merge to this need to be on the same granularity (user id). 

members_v3.csv contains member details and can be joined directly. user_logs_v2.csv contains listening behaviour information per user log and must be aggregated to user level to be merged. The same applies to transactions_v2.csv which includes data about payment transactions in march.

## Join member attributes to target data

In [1]:
import pandas as pd

# Path to folder containing data files
path = "/Users/fynn/Desktop/ML/KKBOX-Data/"

# Load target & members data
target  = pd.read_csv(path + "train_v2.csv")
members = pd.read_csv(path + "members_v3.csv")

# Join member attributes to target
kkbox_churn = target.merge(members, left_on="msno", right_on="msno", how="left")
kkbox_churn

,msno,is_churn,city,bd,gender,registered_via,registration_init_time
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,male,3.0,20131223.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,male,3.0,20131227.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,0.0,NaN,7.0,20140109.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,female,7.0,20140125.0
...,...,...,...,...,...,...,...
970955,OHnZbu+EVaP+vN7Z+OfT5OMcp90MWFZonmM0o3pb8FY=,0,13.0,28.0,male,7.0,20110831.0
970956,S92bDK//uI6hk3u1vuApro0qJBQOToBozZ7lL1yTC+w=,0,1.0,0.0,NaN,7.0,20110903.0
970957,eUa3xo16vpAjr43Cjlb6Kjf1NTILYyJIkBayJQdXWnw=,0,1.0,0.0,NaN,7.0,20110905.0
970958,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0,5.0,25.0,female,9.0,20110905.0


There are 970,960 users in this dataset

## Extract and join relevant user listening behaviour attributes

In [2]:
# Load user log data
userlog = pd.read_csv(path + "user_logs_v2.csv")

# Get march count of logs per user
userlog_last_month_count = userlog[["msno", "date"]].groupby(["msno"]).count()
userlog_last_month_count.columns = ["count_userlogs_last_month"]

# Get march sum (whole data frame) values for all attributes
userlog_last_month_grouped = userlog.drop(["date"], axis=1).groupby(["msno"]).sum()
userlog_last_month_grouped.columns = ["num_25_last_month", "num_50_last_month", "num_75_last_month", "num_985_last_month", 
                              "num_100_last_month", "num_unq_last_month", "total_secs_last_month"]

# Get last march week sum (whole data frame) values for all attributes
userlog_last_week = userlog[userlog["date"] >= 20170324]
userlog_last_week_grouped = userlog_last_week.drop(["date"], axis=1).groupby(["msno"]).sum()
userlog_last_week_grouped.columns = ["num_25_last_week", "num_50_last_week", "num_75_last_week", "num_985_last_week", 
                                     "num_100_last_week", "num_unq_last_week", "total_secs_last_week"]

# IDEA: time_incr_last_month
# Take the total listening time for each week and run a linear regression for every user on these four datapoints.
# The slope of this regression will be used as a further attribute indicating wether the users listening time has
# increased (positive slope) or decreased (negative slope) over the last month (significance is disregarded)

# Merge all extracted features to kkbox_churn data frame
kkbox_churn_ext = kkbox_churn.merge(userlog_last_month_count, left_on="msno", right_index=True, how="left")\
                             .merge(userlog_last_month_grouped, left_on="msno", right_index=True, how="left")\
                             .merge(userlog_last_week_grouped, left_on="msno", right_index=True, how="left")
kkbox_churn_ext

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,count_userlogs_last_month,num_25_last_month,num_50_last_month,...,num_100_last_month,num_unq_last_month,total_secs_last_month,num_25_last_week,num_50_last_week,num_75_last_week,num_985_last_week,num_100_last_week,num_unq_last_week,total_secs_last_week
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,11.0,186.0,23.0,...,318.0,348.0,80598.557,10.0,2.0,2.0,0.0,23.0,31.0,5938.416
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,male,3.0,20131223.0,6.0,0.0,4.0,...,26.0,30.0,6986.509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,male,3.0,20131227.0,20.0,239.0,57.0,...,205.0,432.0,67810.467,33.0,18.0,9.0,4.0,44.0,76.0,14466.966
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,0.0,NaN,7.0,20140109.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,female,7.0,20140125.0,15.0,9.0,7.0,...,962.0,548.0,239882.241,1.0,1.0,2.0,3.0,426.0,253.0,106838.311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970955,OHnZbu+EVaP+vN7Z+OfT5OMcp90MWFZonmM0o3pb8FY=,0,13.0,28.0,male,7.0,20110831.0,24.0,48.0,7.0,...,318.0,291.0,77031.435,28.0,1.0,0.0,0.0,18.0,40.0,5258.431
970956,S92bDK//uI6hk3u1vuApro0qJBQOToBozZ7lL1yTC+w=,0,1.0,0.0,NaN,7.0,20110903.0,5.0,43.0,8.0,...,22.0,54.0,8697.066,10.0,6.0,5.0,4.0,14.0,19.0,5596.102
970957,eUa3xo16vpAjr43Cjlb6Kjf1NTILYyJIkBayJQdXWnw=,0,1.0,0.0,NaN,7.0,20110905.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
970958,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0,5.0,25.0,female,9.0,20110905.0,23.0,33.0,10.0,...,336.0,309.0,79499.246,4.0,0.0,1.0,2.0,56.0,47.0,13244.663


We enlarged the dataset by 15 features about user behaviour (last month and week) extracted from the user logs file

## Extract and join relevant user transaction attributes

In [14]:
# Load transaction data
transact = pd.read_csv(path + "transactions_v2.csv")

# Get average length of membership plan, plan list price & actual amount paid
transact_avg = transact[["msno", "payment_plan_days", "plan_list_price", "actual_amount_paid"]].groupby(["msno"]).mean()
transact_avg.columns = ["avg_payment_plan_days", "avg_plan_list_price", "avg_actual_amount_paid"]

# Get most recently used payment method and auto renewal mode
transact_last = transact[["msno", "payment_method_id", "is_auto_renew"]].groupby(["msno"]).last()
transact_last.columns = ["last_payment_method_id", "last_is_auto_renew"]

# Get the days left in membership after last transaction
transact_memberdays_left = transact[["msno", "transaction_date", "membership_expire_date"]].groupby(["msno"]).last() #apply(lambda x: x["membership_expire_date"] - x["transaction_date"])
transact_memberdays_left["memberdays_left"] = transact_memberdays_left["membership_expire_date"] - transact_memberdays_left["transaction_date"]
transact_memberdays_left = transact_memberdays_left["memberdays_left"]

# Merge all extracted features to kkbox_churn data frame
kkbox_churn_ext2 = kkbox_churn_ext.merge(transact_avg, left_on="msno", right_index=True, how="left")\
                                  .merge(transact_last, left_on="msno", right_index=True, how="left")\
                                  .merge(transact_memberdays_left, left_on="msno", right_index=True, how="left")
kkbox_churn_ext2

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,count_userlogs_last_month,num_25_last_month,num_50_last_month,...,num_985_last_week,num_100_last_week,num_unq_last_week,total_secs_last_week,avg_payment_plan_days,avg_plan_list_price,avg_actual_amount_paid,last_payment_method_id,last_is_auto_renew,memberdays_left
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,11.0,186.0,23.0,...,0.0,23.0,31.0,5938.416,NaN,NaN,NaN,NaN,NaN,NaN
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,male,3.0,20131223.0,6.0,0.0,4.0,...,NaN,NaN,NaN,NaN,30.0,180.0,180.0,36.0,0.0,100.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,male,3.0,20131227.0,20.0,239.0,57.0,...,4.0,44.0,76.0,14466.966,75.0,150.0,150.0,15.0,0.0,301.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,0.0,NaN,7.0,20140109.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30.0,149.0,149.0,41.0,1.0,19600.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,female,7.0,20140125.0,15.0,9.0,7.0,...,3.0,426.0,253.0,106838.311,30.0,99.0,99.0,41.0,1.0,602.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970955,OHnZbu+EVaP+vN7Z+OfT5OMcp90MWFZonmM0o3pb8FY=,0,13.0,28.0,male,7.0,20110831.0,24.0,48.0,7.0,...,0.0,18.0,40.0,5258.431,30.0,149.0,149.0,40.0,1.0,99.0
970956,S92bDK//uI6hk3u1vuApro0qJBQOToBozZ7lL1yTC+w=,0,1.0,0.0,NaN,7.0,20110903.0,5.0,43.0,8.0,...,4.0,14.0,19.0,5596.102,30.0,99.0,99.0,41.0,1.0,100.0
970957,eUa3xo16vpAjr43Cjlb6Kjf1NTILYyJIkBayJQdXWnw=,0,1.0,0.0,NaN,7.0,20110905.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30.0,99.0,99.0,41.0,1.0,100.0
970958,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0,5.0,25.0,female,9.0,20110905.0,23.0,33.0,10.0,...,2.0,56.0,47.0,13244.663,30.0,149.0,149.0,39.0,1.0,185.0


We enlarged the dataset by 6 features about user transactions extracted from the transactions file

## Create and save clean data file

In [17]:
kkbox_churn_ext2.to_csv("kkbox_churn.csv" ,index=False)

The output file of this phase which will be used from the Data Preprocessing stage onwards includes the following features:

![alt text](media/kkbox_data_understanding_output.png)